# Padding a message

In [2]:
from crypto import bytes_to_bin , bytes_to_hex

In [15]:
message = b'Block ciphers are more powerfull than the stream ciphers'
msg_bin = bytes_to_bin(message)
msg_hex  = bytes_to_hex(message)

print(f"message : \t{message}\n")
print(f'binary :\t{msg_bin[2:]}\n')
print(f'hexadicimal :\t{msg_hex[2:]}')


message : 	b'Block ciphers are more powerfull than the stream ciphers'

binary :	0100001001101100011011110110001101101011001000000110001101101001011100000110100001100101011100100111001100100000011000010111001001100101001000000110110101101111011100100110010100100000011100000110111101110111011001010111001001100110011101010110110001101100001000000111010001101000011000010110111000100000011101000110100001100101001000000111001101110100011100100110010101100001011011010010000001100011011010010111000001101000011001010111001001110011

hexadicimal :	426c6f636b206369706865727320617265206d6f726520706f77657266756c6c207468616e207468652073747265616d2063697068657273


In [17]:
print(f"the message length: {len(message)} (bytes), {len(msg_bin)-2} (binary)")

the message length: 56 (bytes), 448 (binary)


we will use a 16bytes block, so we will check if 56 is multiple of 16, if "Yes" then we are fine if not then we are going to use the padding using the PKCS7 method 

In [118]:
def PKCS7(m,block_size_byte=16): 
    rest = block_size_byte - len(m) %block_size_byte
    pad  =  bytes([rest for _ in range(rest)])
    return m +pad

    

In [120]:
PKCS7(b"Block ciphers are more powerfull than the stream",15)

b'Block ciphers are more powerfull than the stream\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c'

We can find the PKCS7 pre-implemented in the cryptography.hazmat.primitives package

In [122]:
from cryptography.hazmat.primitives import padding

In [194]:
block_size = 16

#instanciating the object 
padder = padding.PKCS7(block_size *8 ).padder()
 # the block size must be in bits
padded_message = padder.update(message) + padder.finalize()
# padder.finalize() is to tell the padder that the message is finished, if we want to add another messages we must includes them before padder.finalize()


In [195]:
padded_message

b'Block ciphers are more powerfull than the stream ciphers\x08\x08\x08\x08\x08\x08\x08\x08'

# Using AES 

In [207]:
from cryptography.hazmat.primitives.ciphers import Cipher , algorithms, modes
from cryptography.hazmat.backends import  default_backend
# the default_backend use the OpelSSl package
import os 
secret_key = os.urandom(32)
cipher = Cipher(algorithms.AES(secret_key),modes.ECB(),backend=default_backend())
encryptor = cipher.encryptor()
decryptor = cipher.decryptor()




In [208]:
# we use the padded message from the PKCS7 method
cipher_text = encryptor.update(padded_message)  
cipher_text1 = encryptor.update(padded_message)
#encryptor.finalize() for telling the encryptor that: this is the end of the message
plain_text = decryptor.update(cipher_text) + decryptor.finalize()
print(f'the first cipher text : {cipher_text}')
print(f'the second cipher text : {cipher_text1}')

#the .finalize() is for the decryptor and for the encryptor it's created  when we instanciate the object form cipher.encryptor()

the first cipher text : b'<\xd9\xe8\x12nh0\x03u\xf2\xaeK1\xafG\xc8\x15-\xdc\xc2\x0f\\\x94\x90/T\xef\xbf\x03B\x8a\xed\xce\xc0TX\x96\xc7\xa5\xd2\x1ew/\xea\xe2\xc5\r\xbc\xff\x8b\xcb\xc29d\xf9\xaf\xd9\xa5 \x93\xf7\xeeM\x8d'
the second cipher text : b'<\xd9\xe8\x12nh0\x03u\xf2\xaeK1\xafG\xc8\x15-\xdc\xc2\x0f\\\x94\x90/T\xef\xbf\x03B\x8a\xed\xce\xc0TX\x96\xc7\xa5\xd2\x1ew/\xea\xe2\xc5\r\xbc\xff\x8b\xcb\xc29d\xf9\xaf\xd9\xa5 \x93\xf7\xeeM\x8d'


In [205]:
print(f"The cipher text is :\n\t {cipher_text}")
print(f"The plain text is :\n\t {plain_text}")

The cipher text is :
	 b'x\xa6\x96\x1b\x9an\xbeJ\xd8\xd3m\x05|\xdc\x13kl\xdcZ\xe4\xed~E\xcb\x15\xecI\x14\x11\xf5D\xbaXC\xda\xe7l\x8f\xb8\x80\xd1\x85\xe1Ct\t\x1dW\x03\xcfQ&y\xb7\xf1T\xcde\xeb\xd6\x92\x00\x0fp'
The plain text is :
	 b'Block ciphers are more powerfull than the stream ciphers\x08\x08\x08\x08\x08\x08\x08\x08'


# Mode d'Opération

In [219]:
secret_key = os.urandom(32)
cipher = Cipher(algorithms.AES(secret_key),modes.ECB(),default_backend())
encryptor = cipher.encryptor()
decryptor = cipher.decryptor()


In [215]:
padded_message

b'Block ciphers are more powerfull than the stream ciphers\x08\x08\x08\x08\x08\x08\x08\x08'

In [216]:
new_message = padded_message + padded_message
cipher_text = encryptor.update(new_message) + encryptor.finalize()

print(cipher_text[0:len(padded_message)])
print(cipher_text[len(padded_message):])

b'\xcf \xc6\x1c\x13t\xd9;\xf5\xb8\x016Y\xec|\x02\x81n\xbb\xf9a\x84\xc7\x9dC\xd4\x96<\x08!\x01\\\x8d\x01\xfc\x94\x06\xcf\xf4\xf4\xc3\xbd\x8by\x86\x87SX\x9e~OA\xa5p\xf1\xc3\xc7oND\xe0\xb9\xc8`'
b'\xcf \xc6\x1c\x13t\xd9;\xf5\xb8\x016Y\xec|\x02\x81n\xbb\xf9a\x84\xc7\x9dC\xd4\x96<\x08!\x01\\\x8d\x01\xfc\x94\x06\xcf\xf4\xf4\xc3\xbd\x8by\x86\x87SX\x9e~OA\xa5p\xf1\xc3\xc7oND\xe0\xb9\xc8`'


We note that we have the same cipher text and this is due to the ECB mode 

In [222]:
secret_key = os.urandom(32)
iv = os.urandom(16)

cipher = Cipher(algorithms.AES(secret_key),modes.CBC(iv),default_backend())
encryptor = cipher.encryptor()
decryptor = cipher.decryptor()


In [223]:
new_message = padded_message + padded_message
cipher_text = encryptor.update(new_message) + encryptor.finalize()

print(cipher_text[0:len(padded_message)])
print("\n")
print(cipher_text[len(padded_message):])

b"\x89\x9b6\x9f0\xaf\x18G=\xf0\x08X:\xc4XuQ(\xf3\xac\x87R\x01(6\x05\r\xc4\x1c\xee\xb3\xda\x92\x13\x14\xd4*\xfb)\xd4\xcaZ\x0f.\x17b\x7f\x93'Id\xf0\xc2\xfd)\xe0~\x87\x03\xe8\x89m\xd7e"


b"\x86#\\\x88g#`0\x8d\x93\xef\xdb\xeb\xaa^Ld\x88\xefY\x9d\xb3\xe0\xea\x1d\xe9\x9f\n\x1f\xd4\xac\xd1\xcf\x94,\xcd\xdc\x8e;\xd5\x88'\xc7,\xf1|f\xe1\x15\x0fG\x88s\x9b\xfe\x8a\x18\x1c\xc6\x8d`--H"


In [224]:
len(cipher_text)

128

# Cipher text size

In [236]:
secret_key = os.urandom(32)
iv = os.urandom(16)
cipher = Cipher(algorithms.AES(secret_key),modes.CBC(iv),default_backend())
block_size = 16
for msg_len in range(128):
    m = str.encode("a"*msg_len)
    padder = padding.PKCS7(block_size * 8 ).padder()
    message_padded = padder.update(m) + padder.finalize()
    encryptor = cipher.encryptor()
    cipher_text = encryptor.update(message_padded) + encryptor.finalize()
#     print(f"message : {m}, padded_message = {message_padded }, cipher text : {cipher_text}")
#     print(f"message len = {len(m)} , message padded len = {len(message_padded)} , cipher text len= {len(cipher_text)} ")
    print(f"message : {m} ({len(m)}), padded_message = {message_padded }")
    
    

message : b'' (0), padded_message = b'\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'
message : b'a' (1), padded_message = b'a\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f'
message : b'aa' (2), padded_message = b'aa\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e'
message : b'aaa' (3), padded_message = b'aaa\r\r\r\r\r\r\r\r\r\r\r\r\r'
message : b'aaaa' (4), padded_message = b'aaaa\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c'
message : b'aaaaa' (5), padded_message = b'aaaaa\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b'
message : b'aaaaaa' (6), padded_message = b'aaaaaa\n\n\n\n\n\n\n\n\n\n'
message : b'aaaaaaa' (7), padded_message = b'aaaaaaa\t\t\t\t\t\t\t\t\t'
message : b'aaaaaaaa' (8), padded_message = b'aaaaaaaa\x08\x08\x08\x08\x08\x08\x08\x08'
message : b'aaaaaaaaa' (9), padded_message = b'aaaaaaaaa\x07\x07\x07\x07\x07\x07\x07'
message : b'aaaaaaaaaa' (10), padded_message = b'aaaaaaaaaa\x06\x06\x06\x06\x06\x06'
message : b'aaaaaaaaaaa' (11)

we see that the size of the cipher text is the same as the size of the padded message and that's because the encryption is by blocks 

and we see also that when the message len reaches 16 the padded message go to 32 and that because the rest will be 16 (block_size - len(m) % block_size=16) then we add 16 bytes.

All the lengths are a multiples of 16 (block size)

15